# Part 2

## Creating New Metaclasses

In [1]:
from slide5_dynamic_class import MyClass, MyMixin

from slide9_better_repr import better_repr_type

def __init__(self, x):
    self.x = x

In [2]:
MySubClassWithRepr = better_repr_type(
    'MySubClassWithRepr',  # name
    (MyClass, MyMixin),  # bases
    {'__init__': __init__},   # attributes / methods
)

In [3]:
MySubClassWithRepr

MySubClassWithRepr(MyClass, MyMixin)

## Using Metaclasses in "Normal" Classes


In [4]:
from slide5_dynamic_class import MyClass

from slide9_better_repr import better_repr_type

class MySubClassWithRepr2(MyClass, metaclass=better_repr_type):
    def __init__(self, x):
        self.x = x


In [5]:
MySubClassWithRepr2

MySubClassWithRepr2(MyClass)

## Class Decorators


In [6]:
from slide20_meta_alternatives import *

Waterfowl['Duck']

** __init__ in metaclass for Waterfowl
** registering Duck
** __init__ in metaclass for Duck
** verifying: Duck
** verified: Duck


<Class for making 'Duck'>

In [7]:
class Cat(Duck):
    def quack(self):
        print("meow!")

** registering Cat
** __init__ in metaclass for Cat


In [8]:
@check_waterfowl
class Dog(Duck):
    def quack(self):
        print("woof, woof!")

** registering Dog
** __init__ in metaclass for Dog
** verifying: Dog


RuntimeError: What a quack! Dog is not quacky enough.